# Which Countries Love America the Most?


## Convert DATA

In [1]:
from tld import get_tld

def get_domain(url):
    res = get_tld(url, as_object=True)
    return res.tld

In [5]:
import bz2
import json
import os.path

path_to_file = './input/quotes-2019-nytimes.json.bz2'
path_to_out = './input/quotes-2019-nytimes-domains.json.bz2'

if os.path.isfile(path_to_out):
    print("The output file already exists.")
else:
    with bz2.open(path_to_file, 'rb') as s_file:
        with bz2.open(path_to_out, 'wb') as d_file:
            for instance in s_file:
                instance = json.loads(instance) # loading a sample
                urls = instance['urls'] # extracting list of links
                domains = []
                for url in urls:
                    tld = get_domain(url)
                    domains.append(tld)
                instance['domains'] = domains # updating the sample with domain name
                d_file.write((json.dumps(instance)+'\n').encode('utf-8')) # writing in the new file

The output file already exists.


## Load DATA

In [14]:
from ast import literal_eval
import pandas as pd

with open('input/data') as f:
    data = f.readlines()
    data = [literal_eval(x.strip()) for x in data]


data = pd.DataFrame(data)
data.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,domains
0,2019-04-17-024782,"It is not a low-income immigration,",James Fisher,"[Q16213953, Q20707104, Q43143598, Q58886302, Q...",2019-04-17 13:31:18,1,"[[James Fisher, 0.7475], [None, 0.2525]]",[https://www.nytimes.com/2019/04/17/realestate...,E,[com]
1,2019-04-02-001128,a champion figure skater switching to roller s...,John Updike,[Q105756],2019-04-02 14:58:33,2,"[[John Updike, 0.5856], [None, 0.4144]]",[https://www.nytimes.com/2019/04/02/opinion/vl...,E,"[com, com]"
2,2019-05-09-055187,It makes it much more difficult for him to mak...,None,[],2019-05-09 18:11:29,1,"[[None, 0.6493], [President Bill Clinton, 0.27...",[http://mobile.nytimes.com/2019/05/09/world/as...,E,[com]
3,2019-10-31-056366,"It puts me in a predicament,",Xavier Becerra,[Q1855840],2019-10-31 16:45:15,3,"[[Xavier Becerra, 0.9065], [None, 0.0909], [St...",[http://www.nytimes.com/2019/10/31/technology/...,E,"[com, com, com]"
4,2019-01-04-001792,A Pile of Leaves.,None,[],2019-01-04 10:00:07,1,"[[None, 0.8737], [Jason Fulford, 0.1263]]",[https://www.nytimes.com/2019/01/04/books/revi...,E,[com]


In [27]:
keywords = ["US", "U.S", "USA", "United States"]
mask = data.quotation.apply(lambda x: any(word in x for word in keywords))
US_data = data[mask]

print("Number of filtered quotes {} out of {}".format(len(US_data), len(data)))
US_data[['quotation', 'speaker']].head()

Number of filtered quotes 3263 out of 207527


,quotation,speaker
166,"If there was any merit to Macron's interview, ...",Emmanuel Macron
203,Don't think the U.S. is going to protect you f...,Barnett Rubin
337,This report's central conclusion is that the U...,None
385,Alfred E. Neuman cannot become president of th...,President Donald Trump
510,A strong U.S.-Philippines alliance deters aggr...,None


## Load Speakers


In [31]:
speaker_df = pd.read_parquet("input/speaker_attributes.parquet")
speaker_df.head()

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[Q30],[Q6581097],1395142029,None,None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]",item,"[Q329646, Q682443, Q33203]"
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Q29],[Q6581097],1391704596,None,None,[Q1028181],None,None,Q297,Diego Velázquez,None,item,None


### Drop Quotes with None speakers


In [52]:
drop_mask = US_data[US_data.speaker == "None"].index
US_data = US_data.drop(drop_mask)
print("{} quotes are removed".format(len(drop_mask)))


1295 quotes are removed


### Add nationality to the main df

In [58]:
US_data["speaker_id"] = US_data.qids.map(lambda x: x[0])
US_data.head()

speaker_df = speaker_df[['id', 'nationality']]

US_merged = pd.merge(US_data, speaker_df, left_on=['speaker_id'], right_on=['id'], how='inner')
print("Before : {}, After : {}".format(len(US_data), len(US_merged)))
US_merged.head()


Before : 1968, After : 1952


,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,domains,speaker_id,id,nationality
0,2019-12-02-036754,"If there was any merit to Macron's interview, ...",Emmanuel Macron,[Q3052772],2019-12-02 07:55:07,1,"[[Emmanuel Macron, 0.6888], [None, 0.1613], [I...",[http://www.nytimes.com/2019/12/02/world/europ...,E,[com],Q3052772,Q3052772,[Q142]
1,2019-10-24-005377,And I found out via a tweet that the U.S. had ...,Emmanuel Macron,[Q3052772],2019-10-24 07:00:12,1,"[[Emmanuel Macron, 0.8426], [None, 0.1444], [M...",[https://www.nytimes.com/2019/10/24/world/euro...,E,[com],Q3052772,Q3052772,[Q142]
2,2019-10-24-040910,"I understood that we were together in NATO, th...",Emmanuel Macron,[Q3052772],2019-10-24 07:00:12,1,"[[Emmanuel Macron, 0.8401], [None, 0.129], [Ma...",[https://www.nytimes.com/2019/10/24/world/euro...,E,[com],Q3052772,Q3052772,[Q142]
3,2019-08-23-030461,"I know the points of disagreement with the U.S.,",Emmanuel Macron,[Q3052772],2019-08-23 15:45:54,8,"[[Emmanuel Macron, 0.7694], [None, 0.189], [Do...",[http://nytimes.com/2019/08/23/us/politics/g7-...,E,"[com, com, com, com, com, com]",Q3052772,Q3052772,[Q142]
4,2019-11-07-121863,We should reassess the reality of what Nato is...,Emmanuel Macron,[Q3052772],2019-11-07 10:58:08,11,"[[Emmanuel Macron, 0.6947], [None, 0.1943], [P...",[http://www.breitbart.com/news/nato-allies-cla...,E,"[com, com, com, com, com, com, co.za, com, com...",Q3052772,Q3052772,[Q142]


## Get Sentiments

In [59]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/alireza/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [71]:
sia = SentimentIntensityAnalyzer()
US_merged["sentiment"] = US_merged.quotation.apply(lambda x: sia.polarity_scores(x)["compound"])
US_merged.head()


,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,domains,speaker_id,id,nationality,sentiment
0,2019-12-02-036754,"If there was any merit to Macron's interview, ...",Emmanuel Macron,[Q3052772],2019-12-02 07:55:07,1,"[[Emmanuel Macron, 0.6888], [None, 0.1613], [I...",[http://www.nytimes.com/2019/12/02/world/europ...,E,[com],Q3052772,Q3052772,[Q142],0.6249
1,2019-10-24-005377,And I found out via a tweet that the U.S. had ...,Emmanuel Macron,[Q3052772],2019-10-24 07:00:12,1,"[[Emmanuel Macron, 0.8426], [None, 0.1444], [M...",[https://www.nytimes.com/2019/10/24/world/euro...,E,[com],Q3052772,Q3052772,[Q142],0.0000
2,2019-10-24-040910,"I understood that we were together in NATO, th...",Emmanuel Macron,[Q3052772],2019-10-24 07:00:12,1,"[[Emmanuel Macron, 0.8401], [None, 0.129], [Ma...",[https://www.nytimes.com/2019/10/24/world/euro...,E,[com],Q3052772,Q3052772,[Q142],0.0000
3,2019-08-23-030461,"I know the points of disagreement with the U.S.,",Emmanuel Macron,[Q3052772],2019-08-23 15:45:54,8,"[[Emmanuel Macron, 0.7694], [None, 0.189], [Do...",[http://nytimes.com/2019/08/23/us/politics/g7-...,E,"[com, com, com, com, com, com]",Q3052772,Q3052772,[Q142],-0.3612
4,2019-11-07-121863,We should reassess the reality of what Nato is...,Emmanuel Macron,[Q3052772],2019-11-07 10:58:08,11,"[[Emmanuel Macron, 0.6947], [None, 0.1943], [P...",[http://www.breitbart.com/news/nato-allies-cla...,E,"[com, com, com, com, com, com, co.za, com, com...",Q3052772,Q3052772,[Q142],0.6597
